In [12]:
import sys
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, '../src')

%load_ext autoreload
%autoreload 2

import torch
import torch_geometric
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt


from torch_geometric.datasets import TUDataset
from preprocessing import data_transformation
from similarity import calculate_similarity_matrix

from model import GCN
import copy

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
dataset = TUDataset(root='datasets/', name='MUTAG')
torch.manual_seed(1234)
dataset = dataset.shuffle()

## Preprocessing

#### Split: Train test validation

```train_dataset```: for training model<br/>
```val_dataset```: evaluate model for hyperparameter tunning<br/>
```test_dataset```: testing model after complete training<br/>

In [14]:
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data

## Building Model

In [15]:
from torch_geometric.nn import GCNConv
from torch_geometric.nn import GraphSAGE
from torch_geometric.nn import GraphConv
from torch_geometric.nn import GINConv
# from torch_geometric.nn import GINConv
from torch.nn import Linear, Sequential, ReLU, BatchNorm1d

from torch_geometric.nn import global_mean_pool
from torch_geometric.nn import global_add_pool
import torch.nn.functional as F

#### Base

In [18]:
class Base(torch.nn.Module):
    # merging type: o --> complement only, s --> substraction, c --> concatenation
    def __init__(self, dataset, hidden_channels):
        super(Base, self).__init__()
        
        # weight seed
        torch.manual_seed(42)
        nn1 = Sequential(Linear(dataset.num_node_features, hidden_channels), ReLU(), Linear(hidden_channels,hidden_channels))
        self.conv1 = GINConv(nn1)
        self.bn1 = BatchNorm1d(hidden_channels)
        
        nn2 = Sequential(Linear(hidden_channels, hidden_channels), ReLU(), Linear(hidden_channels,hidden_channels))
        self.conv2 = GINConv(nn2)
        self.bn2 = BatchNorm1d(hidden_channels)
        
        
        # classification layer        
        self.lin = Linear(hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index, batch):
        # Embed original
        embedding = F.relu(self.conv1(x, edge_index))
        embedding = self.bn1(embedding)
        embedding = F.relu(self.conv2(embedding, edge_index))
        embedding = self.bn2(embedding)
        
        embedding = global_add_pool(embedding, batch)
        h = self.lin(embedding)
        h = F.relu(h)
        h = F.dropout(h, p=0.3, training=self.training)
        h = self.lin2(h)
        
        return embedding, h

### Experiment Model

In [153]:
from similarity import calculate_similarity_matrix, testt


# AP Clustering
from sklearn.cluster import AffinityPropagation

from torch_geometric.nn import global_mean_pool
from torch_geometric.nn import global_max_pool
from torch_geometric.nn import GINConv, global_add_pool
from torch.nn import Linear, Sequential, ReLU, BatchNorm1d

import torch.nn.functional as F

class Experiment(torch.nn.Module):
    # merging type: o --> complement only, s --> substraction, c --> concatenation
    def __init__(self, dataset, hidden_channels, k = 1):
        super(Experiment, self).__init__()
        
        # save number of subgraphs, default 1
        self.k_subgraph = k
        
        # weight seed
        torch.manual_seed(42)
        nn1 = Sequential(Linear(dataset.num_node_features, hidden_channels), ReLU(), Linear(hidden_channels,hidden_channels))
        self.conv1 = GINConv(nn1)
        self.bn1 = BatchNorm1d(hidden_channels)
        
        nn2 = Sequential(Linear(hidden_channels, hidden_channels), ReLU(), Linear(hidden_channels,hidden_channels))
        self.conv2 = GINConv(nn2)
        self.bn2 = BatchNorm1d(hidden_channels)
        
        # embeddings for subgraph
        self.conv4 = GCNConv(hidden_channels, hidden_channels)
        self.conv5 = GCNConv(hidden_channels, hidden_channels)
        
        # attention layer
        self.query_layer = Linear(hidden_channels,hidden_channels)
        self.key_layer = Linear(hidden_channels,hidden_channels)
        self.value_layer = Linear(hidden_channels,hidden_channels)
        
        # classification layer
        self.lin = Linear(hidden_channels*2, hidden_channels)
        self.lin2 = Linear(hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index, batch, ptr):
        # Embed original
        # embedding = self.conv1(x, edge_index)
        # embedding = embedding.relu()
        # embedding = self.conv2(embedding, edge_index)
                
        embedding = F.relu(self.conv1(x, edge_index))
        embedding = self.bn1(embedding)
        embedding = F.relu(self.conv2(embedding, edge_index))
        embedding = self.bn2(embedding)
        
        # generate subgraph based on embeddings
        feature_emb = embedding.detach()
        
        subgraph_edge_index, communities, S, batch_communities = self.subgraph_generator(feature_emb, edge_index, batch, ptr)
        
        
        subgraph_embedding = self.conv4(embedding, subgraph_edge_index)
        subgraph_embedding = subgraph_embedding.relu()
        subgraph_embedding = self.conv5(subgraph_embedding, subgraph_edge_index)
        
        # apply readout layer/pooling for each subgraphs
        subgraph_pool_embedding = self.subgraph_pooling(subgraph_embedding, communities, batch, ptr, batch_communities)
        # print(len(subgraph_pool_embedding))
        # apply selective (top k) attention
        topk_subgraph_embedding = self.selectk_subgraph(embedding, subgraph_pool_embedding, self.k_subgraph)
        
        # readout layer for original embedding
        embedding = global_mean_pool(embedding, batch)
                
        combined_embeddings = torch.cat((embedding, topk_subgraph_embedding.view(len(embedding), -1)), 1)
        
        
        # h = F.dropout(combined_embeddings, p=0.3, training=self.training)
        h = self.lin(combined_embeddings)
        h = F.relu(h)
        h = F.dropout(h, p=0.3, training=self.training)
        h = self.lin2(h)
        
        return combined_embeddings, h, S, communities, topk_subgraph_embedding.view(len(embedding), -1)
    
    # checked
    def selectk_subgraph(self, embs, sub_embs, k = 1):
        # calculate attention and select top k subgraph
        
        topk_subgraphs_all = []

        for i, (emb, sub_emb) in enumerate(zip(embs, sub_embs)):
            sub = torch.tensor(sub_emb)
            sub = sub.to(torch.float32)

            # transform
            query = self.query_layer(emb)
            key = self.key_layer(sub)
            value = self.value_layer(sub)

            # att score
            attention_score = torch.matmul(query, key.transpose(0,1))
            attention_weight = F.softmax(attention_score, dim=0)
            
            # select topk
            topk_subgraph_embeddings = None
            
            if (k <= len(sub)):
                topk_scores, topk_indices = torch.topk(attention_weight, k)
                topk_subgraph_embeddings = sub[topk_indices]
            else:
                print('too big')
                
            topk_subgraphs_all.append(topk_subgraph_embeddings)
        
        return torch.stack(topk_subgraphs_all)
    
    
    def subgraph_generator(self, embeddings, batch_edge_index, batch, ptr):
        '''
        Return subgraph_edge_index (edge_index of created subgraph)
        '''
        graph_counter = 0
        edge_index = [[],[]]
        subgraph_edge_index = [[],[]]
        # Gs = []
        sub_created = False
        graph_bound = {}
        all_communities = []
        batch_communities = {}
        S = []

        for i in range(len(ptr)-1):
            graph_bound[i] = [ptr[i].item(), ptr[i+1].item()]
        
        for i, (src, dst) in enumerate(zip(batch_edge_index[0], batch_edge_index[1])):
            lower_bound = graph_bound[graph_counter][0]
            upper_bound = graph_bound[graph_counter][1]
            if ((src >= lower_bound and src < upper_bound) or
                (dst >= lower_bound and dst < upper_bound)):
                
                edge_index[0].append(src - lower_bound)
                edge_index[1].append(dst - lower_bound)
            else:
                sub_created = True
                
            if (i == len(batch_edge_index[0]) - 1) or sub_created:
                sub_created = False
                
                embs = []
                # make new graph
                for i, (b, emb) in enumerate(zip(batch, embeddings)):
                    if (b == graph_counter):
                        embs.append(emb)
                
                G = data_transformation(edge_index, embs)
                # dont need this at the moment
                # Gs.append(G)
                
                # Calculate similarity matrix
                S = calculate_similarity_matrix(G)
                
                # AP Clustering        
                clustering = AffinityPropagation(affinity='precomputed', damping=0.8, random_state=42, convergence_iter=15, max_iter=1000)
                clustering.fit(S)
                
                
                # Get community
                communities = {}
                for lab in clustering.labels_:
                    communities[lab] = []
                    all_communities.append(lab)
                for nd, clust in enumerate(clustering.labels_):
                    communities[clust].append(nd)
                
                edge_index = [[],[]]
                batch_communities[graph_counter] = communities
                
                graph_counter+=1
                
                # Make subgraph edge_index
                for c in communities:
                    w = G.subgraph(communities[c])
                    for sub in w.edges:
                        subgraph_edge_index[0].append(sub[0] + lower_bound)
                        subgraph_edge_index[1].append(sub[1] + lower_bound)
                
        
        # print('batch communities', batch_communities)
        return torch.tensor(subgraph_edge_index), all_communities, S, batch_communities
    
        
    # check autograd (done)
    def subgraph_pooling(self, embeddings, communities, batch, ptr, batch_communities, pool_type = 'mean'):
        # batch communities: batch (or graph in this batch) -> communities -> member        
        all_emb_pool = []
        
        # LOOP THROUGH BATCH
        for b in batch_communities:
            
            # initialize array
            emb_pool = [None] * len(batch_communities[b])
            for comm in batch_communities[b]:
                emb_temp = []

                for member in batch_communities[b][comm]:
                    index_used = member + ptr[b].item()
                    emb_temp.append(embeddings[index_used])

                # Pooling per sub graph using PyTorch
                emb_temp_tensor = torch.stack(emb_temp)
                if pool_type == 'mean': # mean pool
                    emb_pool[comm] = torch.mean(emb_temp_tensor, dim=0)
                elif pool_type == 'add': # add pool
                    emb_pool[comm] = torch.sum(emb_temp_tensor, dim=0)
                else:
                    print('TODO: fill later')
                    
            all_emb_pool.append(torch.stack(emb_pool))
        return all_emb_pool

# experiment = Experiment(dataset, 64)
# emb, h, S, communities, sub_emb = experiment(batch1.x, batch1.edge_index, batch1.batch, batch1.ptr)

In [22]:
def train_base(model, loader, experiment_mode=False):
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    
    model.train()
    total_loss = 0
    
    for data in loader:
        optimizer.zero_grad()
        if experiment_mode:
            emb, h, S, communities, sub_emb = model(data.x, data.edge_index, data.batch, data.ptr)
        else:
            emb, h = model(data.x, data.edge_index, data.batch)
        loss = criterion(h, data.y)
        total_loss += loss.item()
        
        loss.backward()
        optimizer.step()

    return loss / len(loader)

@torch.no_grad()
def test_base(model, loader, experiment_mode=False):
    model.eval()
    correct = 0
    for data in loader:
        if experiment_mode:
            emb, h, S, communities, sub_emb = model(data.x, data.edge_index, data.batch, data.ptr)
        else:
            emb, h = model(data.x, data.edge_index, data.batch)
        pred = h.argmax(dim=1)
        correct += int((pred == data.y).sum())
    return correct/len(loader.dataset)

In [23]:
def expTrain(train_loader, val_loader, test_loader, epoch = 10, fold=0):
    import warnings
    warnings.filterwarnings("ignore", category=UserWarning) 

    num_hidden_layer = 128
    experiment = Experiment(dataset, num_hidden_layer)
    loss_history = []
    train_acc_history = []
    val_acc_history = []
    test_acc_history = []
    
    # early stop
    early_stopping_patience = 20
    best_val_score = -float("inf")
    epochs_without_improvement = 0
    best_state = None
    
    # Train
    print('process training')
    for _ in range(epoch):
        if (fold == 6):
            # continue training
            if (_+1 < 16):
                # print(f'im in {fold} P{_}')
                continue
            elif _+1 == 16:
                print('model has been trained till epoch 15, skipping--')
                best_state = torch.load("model-history/GIN-MUTAG/6.experiment_best_model-gin_data-mutag.pth")
                experiment.load_state_dict(best_state)
                
                last_acc = test_base(experiment, test_loader, True)
                best_val_score = round(test_base(experiment, val_loader, True), 5)
                print("last val acc", best_val_score, "last test acc", last_acc)
            
        loss = round(train_base(experiment, train_loader, True).item(), 5)
        train_acc = round(test_base(experiment, train_loader, True), 5)
        val_acc = round(test_base(experiment, val_loader, True), 5)
        test_acc = test_base(experiment, test_loader, True)
        
        loss_history.append(loss)
        train_acc_history.append(train_acc)
        val_acc_history.append(val_acc)
        test_acc_history.append(test_acc)
        
        print(f'epoch {_+1}; loss: {loss}; train_acc: {train_acc}; val_acc: {val_acc}; test_acc: {test_acc}')
        
        # early stop
        if (val_acc > best_val_score):
            best_val_score = val_acc
            epochs_without_improvement = 0
            
            print('best found, save model')
            # save model
            torch.save(experiment.state_dict(), "model-history/"+str(fold)+".experiment_best_model-gin_data-mutag.pth")
            best_state = copy.deepcopy(experiment.state_dict())
        else:
            epochs_without_improvement += 1
            if (epochs_without_improvement >= early_stopping_patience):
                print('early stop triggered')
                break
                
            

    # Test    
    # Create a new instance of the model for testing
    best_model = Experiment(dataset, num_hidden_layer)
    best_model.load_state_dict(best_state)

    # Test
    test = test_base(best_model, test_loader, True)
    print(f'Accuracy: {test}')

    
    return [loss_history, train_acc_history, val_acc_history, test_acc_history]

# expTrain(train_loader, val_loader, test_loader, epoch = 100)

In [24]:
def baseTrain(train_loader, val_loader, test_loader, epoch = 10, fold=0):
    num_hidden_layer = 128
    base = Base(dataset, num_hidden_layer)
    early_stopping_patience = 20
    best_val_score = -float("inf")
    epochs_without_improvement = 0
    best_state = None
    
    
    
    # Train
    for _ in range(epoch):
        if (fold == 6):
            print('model has been trained')
            best_state = torch.load("model-history/GIN-MUTAG/6.base_best_model-gin_data-mutag.pth")
            break
        
        loss = round(train_base(base, train_loader).item(), 5)
        train_acc = round(test_base(base, train_loader), 5)
        val_acc = round(test_base(base, val_loader), 5)
        
        
        print(f'epoch {_}; loss: {loss}; train_acc: {train_acc}; val_acc: {val_acc}; test: {round(test_base(base, test_loader), 2)}')

        if (val_acc > best_val_score):
            best_val_score = val_acc
            epochs_without_improvement = 0
            
            print('best found, save model')
            # save model
            torch.save(base.state_dict(), "model-history/"+str(fold)+".base_best_model-gin_data-mutag.pth")
            best_state = copy.deepcopy(base.state_dict())
        else:
            epochs_without_improvement += 1
            if (epochs_without_improvement >= early_stopping_patience):
                print('early stop triggered')
                break
                
            
    # Test
    # test = test_base(best, test_loader)
    # print(f'Accuracy: {test}')
    
    # Create a new instance of the model for testing
    best_model = Base(dataset, num_hidden_layer)
    best_model.load_state_dict(best_state)

    # Test
    test = test_base(best_model, test_loader)
    print(f'Accuracy: {test}')


#### Cross validation 10

In [25]:
from sklearn.model_selection import KFold

In [26]:
from torch_geometric.nn import GCNConv
from torch_geometric.nn import GraphSAGE
from torch_geometric.nn import GraphConv
from torch_geometric.nn import GINConv
# from torch_geometric.nn import GINConv
from torch.nn import Linear, Sequential, ReLU, BatchNorm1d

from torch_geometric.nn import global_mean_pool
from torch_geometric.nn import global_add_pool
import torch.nn.functional as F

In [31]:
train_dataset = dataset[:round(len(dataset) * 0.8)]
test_dataset = dataset[round(len(dataset) * 0.8):]
print(train_dataset.y)
print(len(train_dataset.y))
print(test_dataset.y)

tensor([1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0,
        1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1,
        0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0,
        0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1,
        1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1,
        1, 1, 0, 1, 1, 1])
150
tensor([0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1,
        0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1])


In [154]:
# 
train_dataset
test_dataset
k = 10
batch_size = 128

splits = KFold(n_splits=k,shuffle=True,random_state=42)
k_counter = 0
fold_logs = {}
hidden_layer = 128

for fold, (train_idx,val_idx) in enumerate(splits.split(np.arange(len(train_dataset)))):
    if (fold < 6):
        continue
    print(f'Fold {fold}/{k}')
    
    fold_train = []
    for key in train_idx:
        fold_train.append(train_dataset[key])

    fold_val = [] 
    for key in val_idx:
        fold_val.append(train_dataset[key])

    tr = DataLoader(fold_train, batch_size=batch_size, shuffle=True)
    vd = DataLoader(fold_val, batch_size=batch_size, shuffle=True)
    ts = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
    
    base = Base(dataset, hidden_layer)
    base.load_state_dict(torch.load('model-history/GIN-MUTAG/6.base_best_model-gin_data-mutag.pth'))
    expe = Experiment(dataset, hidden_layer)
    expe.load_state_dict(torch.load('model-history/GIN-MUTAG/6.experiment_best_model-gin_data-mutag.pth'))
    # print(base)
    
    
    
    # base()
    ys = []
    base_embs = []
    expe_embs = []
    for data in tr:
        emb, h = base(data.x, data.edge_index, data.batch)
        e_emb, e_h, e_S, e_communities, e_sub_emb = expe(data.x, data.edge_index, data.batch, data.ptr)
        
        emb = emb.detach().tolist()
        e_emb = e_emb.detach().tolist()
        for i, e in enumerate(emb):
            base_embs.append(e)
            expe_embs.append(e_emb[i])
        for y in data.y.detach().tolist():
            ys.append(y)
        
    for data in vd:
        emb, h = base(data.x, data.edge_index, data.batch)
        e_emb, e_h, e_S, e_communities, e_sub_emb = expe(data.x, data.edge_index, data.batch, data.ptr)
        
        emb = emb.detach().tolist()
        e_emb = e_emb.detach().tolist()
        for i, e in enumerate(emb):
            base_embs.append(e)
            expe_embs.append(e_emb[i])
        for y in data.y.detach().tolist():
            ys.append(y)
            
    for data in ts:
        emb, h = base(data.x, data.edge_index, data.batch)
        e_emb, e_h, e_S, e_communities, e_sub_emb = expe(data.x, data.edge_index, data.batch, data.ptr)
        
        emb = emb.detach().tolist()
        e_emb = e_emb.detach().tolist()
        for i, e in enumerate(emb):
            base_embs.append(e)
            expe_embs.append(e_emb[i])
        for y in data.y.detach().tolist():
            ys.append(y)
            
    print('base')
    print(len(base_embs))
    print(base_embs)
    print('expe')
    print(len(expe_embs))
    print(expe_embs)
    print('ys')
    print(len(ys))
    print(ys)
    
    break

Fold 6/10
base
188
[[-0.6857906579971313, -0.7464563846588135, 10.61912727355957, -7.279705047607422, -11.8720703125, 4.652785778045654, -0.3763650059700012, -9.034793853759766, 1.4502309560775757, 0.5296311974525452, -0.7488071918487549, 10.242125511169434, -13.104726791381836, 4.29072380065918, -9.412020683288574, 6.378556251525879, 2.1043896675109863, 2.093977212905884, -1.4448403120040894, -3.861438274383545, -6.003077983856201, -8.046135902404785, 0.61326003074646, -0.8515990972518921, -0.6555241346359253, -8.634533882141113, 1.0352985858917236, -6.714815139770508, 0.9453057646751404, 0.6967378854751587, -5.975179195404053, 2.9685678482055664, 0.2111678123474121, 1.2331877946853638, -12.68269157409668, 12.287862777709961, 7.870121002197266, -1.9699006080627441, 0.002810042817145586, -6.463417053222656, 4.520990371704102, -2.3250699043273926, 0.32932615280151367, -10.718838691711426, -1.3006181716918945, 0.9289432764053345, -5.762284278869629, -4.336251735687256, -11.90020942687988

#### Visualization - TSNE

In [155]:

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd

In [156]:
tsne = TSNE(n_components=2, random_state=42)
X_tsne = tsne.fit_transform(np.array(base_embs))

fig = px.scatter(x=X_tsne[:, 0], y=X_tsne[:, 1], color=['red' if label else 'green' for label in ys])
fig.update_layout(
    title="GIN MUTAG (BASE)",
    xaxis_title="First t-SNE",
    yaxis_title="Second t-SNE",
)
fig.show()

In [157]:
tsne = TSNE(n_components=3, random_state=42)
X_tsne = tsne.fit_transform(np.array(base_embs))
# Create a 3D scatter plot
fig = px.scatter_3d(
    x=X_tsne[:, 0], 
    y=X_tsne[:, 1], 
    z=X_tsne[:, 2],
    color=['red' if label else 'green' for label in ys]
)

# Update layout for 3D plot
fig.update_layout(
    title="GIN MUTAG (BASE)",
    scene=dict(
        xaxis_title="First t-SNE Component",
        yaxis_title="Second t-SNE Component",
        zaxis_title="Third t-SNE Component"
    )
)

fig.show()

In [158]:
tsne = TSNE(n_components=2, random_state=42)
X_tsne = tsne.fit_transform(np.array(expe_embs))
fig = px.scatter(x=X_tsne[:, 0], y=X_tsne[:, 1], color=['red' if label else 'green' for label in ys])
fig.update_layout(
    title="GIN MUTAG (EXPERIMENT)",
    xaxis_title="First t-SNE",
    yaxis_title="Second t-SNE",
)
fig.show()

In [159]:
tsne = TSNE(n_components=3, random_state=42)
X_tsne = tsne.fit_transform(np.array(expe_embs))
# Create a 3D scatter plot
fig = px.scatter_3d(
    x=X_tsne[:, 0], 
    y=X_tsne[:, 1], 
    z=X_tsne[:, 2],
    color=['red' if label else 'green' for label in ys]
)

# Update layout for 3D plot
fig.update_layout(
    title="GIN MUTAG (EXPERIMENT)",
    scene=dict(
        xaxis_title="First t-SNE Component",
        yaxis_title="Second t-SNE Component",
        zaxis_title="Third t-SNE Component"
    )
)

fig.show()